# Circuits à deux qubits

Ces exerices sont issus du livre *__Quantum : un peu de mathématiques pour l'informatique quantique de Arnaud Bodin__*, que je recommande fortement.

L'idée est d'appréhender les circuits à deux quibits en utilisant la bibliothèque Qiskit.

In [39]:
import qiskit
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# importing Qiskit
from qiskit import BasicAer

### Exercice 1 : mise en bouche

L'exercice consiste à implémenter un circuit simple contenant deux qubits et deux mesures.



In [31]:
simulator = BasicAer.get_backend('qasm_simulator')

In [32]:
circuit = qiskit.QuantumCircuit(2,2)

In [33]:
circuit.h(0)
circuit.x(1)
circuit.cx(0,1)
circuit.h(1)
circuit.measure([0,1], [0,1])

In [34]:
import PIL
print(circuit.draw(output="text"))
#img_circuit.show()

     ┌───┐          ┌─┐   
q_0: ┤ H ├──■───────┤M├───
     ├───┤┌─┴─┐┌───┐└╥┘┌─┐
q_1: ┤ X ├┤ X ├┤ H ├─╫─┤M├
     └───┘└───┘└───┘ ║ └╥┘
c: 2/════════════════╩══╩═
                     0  1 


In [35]:
job = qiskit.execute(circuit, simulator, shots=1000)
result = job.result()
counts = result.get_counts(circuit)

In [36]:
print(counts)

{'10': 264, '01': 245, '11': 236, '00': 255}


### Exercice 2 : "montrer" que deux circuits sont équivalents

Cet exercice consiste à montrer l'équivalence entre deux circuit.

Nous Utilisation du simulateur `statevector_simulator` afin d'avoir accès aux états quantiques en plus des mesures 
(Attention, cette pratique est utile pour apprendre mais ne correspond pas à la réalité physique !).

In [94]:

simulator = BasicAer.get_backend('statevector_simulator')

Définissons pour commencer un qubit de norme $1$ : $|\psi_1\rangle$, en choisissant deux nombres complexes au hasard $\alpha_0,\beta_0$, en définissant la norme $n$, puis en définissant $\alpha = \alpha_0 / n, \beta = \beta_0 / n$ puis le qubit $|\psi_1\rangle = \alpha|0\rangle + \beta|1\rangle$.

In [95]:
alpha0 = 3 + 4j
beta0 = 2 - 6j

norme = np.sqrt(abs(alpha0)**2 + abs(beta0)**2)

alpha = alpha0 / norme
beta = beta0 / norme

etat_inital = [alpha, beta]
qubit_initial_1 = qiskit.extensions.Initialize(etat_inital)

Définissons de la même manière un deuxième qubit de norme $1$ : $|\psi_2\rangle$.

In [96]:
delta0 = 1 + 8j
gamma0 = 3 + 5j

norme = np.sqrt(abs(delta0)**2 + abs(gamma0)**2)

delta = delta0 / norme
gamma = gamma0 / norme

etat_inital = [delta, gamma]
qubit_initial_2 = qiskit.extensions.Initialize(etat_inital)


Ecrivons le premier circuit et calculons la valeur du 2-qubit en sortie :

In [97]:
circuit1 = qiskit.QuantumCircuit(2,2)
circuit1.h(0)
circuit1.h(1)
circuit1.cx(0,1)
circuit1.h(0)
circuit1.h(1)

print(circuit1.draw(output="text"))

     ┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ H ├
     ├───┤┌─┴─┐├───┤
q_1: ┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘
c: 2/═══════════════
                    


Calculons la valeur du 2-qubit en sortie :

In [98]:
circuit1.append(qubit_initial_1, [0])
circuit1.append(qubit_initial_2, [1])

job = qiskit.execute(circuit1, simulator)
result = job.result()
coefficients = result.get_statevector()

print(coefficients)

[-0.36151284+0.34904688j  0.62329799+0.1246596j  -0.13712556+0.33658092j
  0.44877456-0.09972768j]


Ecrivons le premier circuit et calculons la valeur du 2-qubit en sortie :

In [99]:
circuit2 = qiskit.QuantumCircuit(2,2)
circuit2.cx(1, 0)

print(circuit2.draw(output="text"))

     ┌───┐
q_0: ┤ X ├
     └─┬─┘
q_1: ──■──
          
c: 2/═════
          


Calculons la valeur du 2-qubit en sortie :

In [100]:
circuit2.append(qubit_initial_1, [0])
circuit2.append(qubit_initial_2, [1])

job = qiskit.execute(circuit2, simulator)
result = job.result()
coeff = result.get_statevector()

print(coeff)

[-0.36151284+0.34904688j  0.62329799+0.1246596j  -0.13712556+0.33658092j
  0.44877456-0.09972768j]


## Conclusion :
Les coefficients du qubit en sortie dans les deux circuits sont identique, donc les deux circuits sont équivalents.